# Collecting Mari dataset locally

In [ ]:
out_path='/Users/yuriypeshkichev/Projects/huawei-nlp/final_project/'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%cd /Users/yuriypeshkichev/Projects/huawei-nlp/final_project/

/Users/yuriypeshkichev/Projects/huawei-nlp/final_project


In [5]:
from cyrillicToLatin import cyrToIPA

In [7]:
import pandas as pd

In [8]:
import os

In [6]:
import re

In [10]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

In [ ]:
# This is to retrieve Mari dataset. Had to stop retreveing because they return OK all the way

i=1
input_txt=[]
target_txt=[]
while True:
    res=requests.get(f"http://marlamuter.com/muter/10_%D1%82%D0%BE%D0%BC%D0%B0%D0%BD/{i}")
    i += 1
    if res.status_code != 200:
        print (res.status_code)
        break
    soup = BeautifulSoup(res.text, "html.parser")
    spans1 = soup.find_all('span', {'class' : 'mutlan1'})
    spans3 = soup.find_all('span', {'class' : 'mutlan3'})
    lines1 = [span.get_text() for span in spans1]
    lines3 = [span.get_text() for span in spans3]
    input_txt.extend(lines1)
    target_txt.extend(lines3)
    if i%100 == 0:
        print (i)

In [49]:
with open(out_path+"mhr-marlamuter.tsv","w") as fw:
    for a,b in zip(input_txt,target_txt):
        fw.write(a+"\t"+b+"\n")

In [19]:
def clean_text(string):
    
    a=re.compile("<.*?>")
    txt=a.sub('',string)
    
    txt = txt.replace("'",'')
    txt = re.sub("[\(\[].*?[\)\]]", "", txt).lower()
    txt = re.sub(r'{[^}]*}','',txt)
       
    
    txt = txt.replace('*', '')
    txt = txt.replace('"', '')
    txt = txt.replace('«', '')
    txt = txt.replace('»', '')
    txt = txt.replace('ʼ', '')
    txt = txt.replace('—','')
    txt = " ".join(txt.split())
    
    txt = txt.replace('ˊ','')
    txt = txt.replace('ˋ','')

    txt = txt.replace("=",'')
    #txt =re.sub(r"\d+", "", txt)
    txt = re.sub(r'(?<=[.,])(?=[^\s])', r' ', txt)

    #txt = re.sub(r'[^\w\s]','',txt)
    txt = re.sub(r'[.,"\'?:!;]', '', txt)
    txt = txt.replace("[",'')
    txt = txt.replace("]",'')
    return txt

In [11]:
corpus = pd.read_csv(out_path+'mhr-marlamuter.tsv',header=None,sep="\t")

In [21]:
corpus1=corpus.dropna()

In [25]:
corpus1=corpus1.reset_index(drop=True)

In [26]:
i=0
crp = open(out_path+"mhr_corpus.tsv","w",encoding='utf8')
while i<len(corpus1):

    string1 = clean_text(corpus1[0][i])
    string1 = cyrToIPA(string1)
    string2 = clean_text(corpus1[1][i])

    # if has_latin(string1):
    #     i+=1
    #     continue
    if string1 and string2:
        crp.write(string1+"\t"+string2+"\n")
    i +=1
crp.close()

In [22]:
len(corpus1)

107575